# Qual a acertividade das previsões publicadas pelo BACEN?

O Banco Central publica semanalmente um boletim contendo as estatísticas calculadas considerando as expectativas de mercado para os principais índices econômicos brasileiros, que contém valores para o ano corrente e os próximos, chamado Relatório Focus.

Os valores apontados no relatório tem como origem diversos agentes econômicos que não fazem parte deste órgão, como instituições financeiras, empresas e universidades.

Estas previsões são impactadas por fatores externos de baixa ou nenhuma previsibilidade como a pandemia de COVID-19, guerra na Ucrânia, resultados de eleições, entre outros, que causam variações em seus números.

Mesmo assim, é válido entender quão acertivas são estas expectativas de mercado, já que elas ajudam a nortear os planejamentos financeiros/estratégicos de diversas empresas e pessoas, nos mais variados ramos de atuação.



In [1]:
# Bibliotecas a serem utilizadas
import pandas as pd
from datetime import datetime as dt
from IPython.display import IFrame
from pybacen.bacen import time_series

# Fonte de dados

## Sistema Expectativas de Mercado

Para captura das previsões de mercado, o BACEN disponibiliza uma plataforma (https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos) que possibilita montar consultas a serem feitas em sua API apenas selecionando o peíodo desejado, e posteriormente os parâmetros.

O endereço base da API é ``https ://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/[codigo_recurso]?$format=json&[Outros Parâmetros]``, considerando os códigos de recursos abaixo.

> ExpectativaMercadoMensais, ExpectativasMercadoTrimestrais, ExpectativasMercadoAnuais, ExpectativasMercadoInflacao12Meses, ExpectativasMercadoTop5Mensais, ExpectativasMercadoTop5Anuais, ExpectativasMercadoInstituicoes

Então, basta usar o link obtido para efetuar a consulta e analisar os dados.

In [2]:

IFrame(
    'https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos', 
    width='100%', 
    height=250
)

In [3]:
IFrame(
    'https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!' \
    '/recursos/ExpectativasMercadoAnuais#eyJmb3JtdWxhcmlvIjp7IiRmb3JtYXQiOiJqc29uIiwiJHRvcCI6MTAwfX0=',
    width='100%',
    height=250
)

## Histórico dos índices

Situação similar ocorre para os dados reais, onde existe uma API chamada BCData/SGS - Sistema Gerenciador de Séries Temporais, conforme exemplo a seguir.

As consultas são feitas utilizando o endereço padrão ``http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={dataInicial}&dataFinal={dataFinal}``.

In [4]:
IFrame(
    'https://dadosabertos.bcb.gov.br/dataset/24363-indice-de-atividade-economica-do-banco-central' \
    '---ibc-br/resource/7d5869ad-b479-47d3-b98f-b99d8d008d5b',
    width='100%',
    height=250
)

Para encontrar o código da série deseja existem dois métodos, diretamente na plataforma do BACEN [``https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries``](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries), ou utilizando a biblioteca `pybacen`.

Na plataforma do BACEN, basta inserir o nome do indicador no campo "Pesquisa textual", avaliar qual indicador mais se adequa a necessidade e usar seu código na API.

<img src="img/pesquisa_indicador_sgs.png" alt="Sistema Gerenciador de Séries Temporais" style="width: 100%;"/>

Na biblioteca ``pybacen`` a pesquisa se torna um pouco mais acertiva, porque é possível pesquisar partes do nome do indicador, bem como escolher a unidade, periodicidade e fonte.

O resultado da pesquisa é um DataFrame do pandas, e nem sempre o nome do indicador (nm_serie) aparecerá completo. Para resolver isto, é possível tirar o limitador de caracteres exibidos no DataFrame (o padrão é 50) utilizando o método ``pd.options.display.max_colwidth = None``.

Despois de realizar a pesquisa, é interessante voltar para o padrão, pra evitar DataFrames muito grandes e de difícil entendimento.

Outro ponto a considerar é o aviso que é exibido a cada pesquisa, informando para confirmar a informação diretamente no site do BACEN.

In [5]:
pd.options.display.max_colwidth = None

In [6]:
time_series.read_bacen_code(search_text='%ipca%12 meses', period='M')

c:\python\lib\site-packages\pybacen\bacen\time_series.py:116: UserWarning:

Check the website: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries



,bacen_code,nm_serie,unit,periodicity,source,special
11820,13522,Índice nacional de preços ao consumidor - amplo (IPCA) - em 12 meses,%,M,IBGE,N


In [7]:
time_series.read_bacen_code(search_text='%meta%selic%')

c:\python\lib\site-packages\pybacen\bacen\time_series.py:116: UserWarning:

Check the website: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries



,bacen_code,nm_serie,unit,periodicity,source,special
405,432,Taxa de juros - Meta Selic definida pelo Copom,% a.a.,D,Copom,N


In [8]:
pd.options.display.max_colwidth = 50

# Captura dos dados e construção das tabelas/DataFrames

Para iniciar a análise da acertividade das previsões é necessário estabelecer quais indicadores e períodos serão utilizados.

Num primeiro momento, serão analisados dois indicadores, o IPCA (inflação) e a Selic (juros), comparando o valor real de fechamento de cada ano, com sua previsão publicada no início deste ano e do anterior, considerando os últimos 20 anos (2001 a 2021, já que 2022 ainda não está fechado no momento da análise).

Por exemplo:

> IPCA dez/2021, comparado com a previsão deste publicada em jan/2021 e jan/2020


## Expectativas de mercado

No caso das Expectativas de Mercado, o resultado da consulta é a relação de todos os indicadores de acordo com a quantidade de linhas determinadas e os parâmetros estabelecidos.

In [9]:
# Definindo os indicadores

indices = ['IPCA', 'Selic']

# Definindo os parâmetros e efetuando a consulta
codigo = 'ExpectativasMercadoAnuais'
link_api =  "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata" \
            f"/{codigo}?$format=json&$select=Indicador,Data,DataReferencia,Media,Mediana," \
            "DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo"


# Capturando os dados e transformando

consulta_api = pd.read_json(link_api)
previsoes_indicadores = pd.json_normalize(consulta_api.value)
previsoes_indicadores.Data = [dt.strptime(previsoes_indicadores.Data[i], '%Y-%m-%d') 
                              for i in range(len(previsoes_indicadores.Data))]
previsoes_indicadores = previsoes_indicadores.astype({'DataReferencia': int})


# Aplicando filtros e criando o df a ser utilizado

filtro =    (previsoes_indicadores.Indicador.isin(indices)) & \
            (previsoes_indicadores.baseCalculo == 0) & \
            (previsoes_indicadores.Data >= '1998-01-01')
previsoes_indicadores_anuais = previsoes_indicadores[filtro]


# Selecionando apenas o primeiro dado divulgado em cada ano

ano_teste = previsoes_indicadores_anuais.Data.min().year
ultimo_ano = previsoes_indicadores_anuais.Data.max().year
lista_datas = []

while ano_teste <= ultimo_ano: 
    filtro_jan =    (previsoes_indicadores_anuais.Data >= f'{ano_teste}-01-01') & \
                    (previsoes_indicadores_anuais.Data <= f'{ano_teste}-01-31')
    menor_data_jan = previsoes_indicadores_anuais.Data[filtro_jan].min()
    lista_datas.append(menor_data_jan)
 
    ano_teste += 1
    
previsoes_indicadores_anuais = previsoes_indicadores_anuais[previsoes_indicadores_anuais.Data.isin(lista_datas)]
previsoes_indicadores_anuais.reset_index(drop=True, inplace=True)


# Criando a coluna com a informação da referência divulgada

def TestaAno():
    lista = []
    for i in range(len(previsoes_indicadores_anuais.Data)):
        if previsoes_indicadores_anuais.Data[i].year == previsoes_indicadores_anuais.DataReferencia[i]:
            lista.append('Inicio ano')
        elif previsoes_indicadores_anuais.Data[i].year == previsoes_indicadores_anuais.DataReferencia[i] - 1:
            lista.append('Ano anterior')
        else:
            lista.append('Fora do range')
    return lista

previsoes_indicadores_anuais['InfoReferencia'] = TestaAno()


# Pivoteando as tabelas e unificando

ipca_tabela_previsao_final = pd.pivot_table(previsoes_indicadores_anuais.query('Indicador == "IPCA"'), 
                                            values= 'Mediana', 
                                            index= 'DataReferencia',
                                            columns= 'InfoReferencia').reset_index()

ipca_tabela_previsao_final = ipca_tabela_previsao_final[['DataReferencia', 'Ano anterior', 'Inicio ano']]
ipca_tabela_previsao_final.columns = ['DataReferencia', 'IPCA ano anterior', 'IPCA inicio do ano']

selic_tabela_previsao_final = pd.pivot_table(previsoes_indicadores_anuais.query('Indicador == "Selic"'), 
                                             values= 'Mediana', 
                                             index= 'DataReferencia',
                                             columns= 'InfoReferencia').reset_index()

selic_tabela_previsao_final = selic_tabela_previsao_final[['DataReferencia', 'Ano anterior', 'Inicio ano']]
selic_tabela_previsao_final.columns = ['DataReferencia', 'Selic ano anterior', 'Selic inicio do ano']
tabela_expectativas = pd.merge(ipca_tabela_previsao_final, selic_tabela_previsao_final, how= 'left', on= 'DataReferencia')

In [10]:
tabela_expectativas.sample(10)

,DataReferencia,IPCA ano anterior,IPCA inicio do ano,Selic ano anterior,Selic inicio do ano
11,2011,4.5000,5.33,10.88,12.25
7,2007,4.5000,4.00,13.50,11.75
23,2023,3.3802,NaN,8.00,NaN
13,2013,5.0000,5.49,10.25,7.25
25,2025,NaN,NaN,NaN,NaN
20,2020,4.0000,3.60,8.00,4.50
4,2004,8.0000,6.00,17.00,13.85
2,2002,3.8100,4.75,12.66,17.00
10,2010,4.5000,4.50,11.50,11.00
21,2021,3.7500,3.32,6.50,3.25


In [11]:
tabela_expectativas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DataReferencia       26 non-null     int64  
 1   IPCA ano anterior    22 non-null     float64
 2   IPCA inicio do ano   23 non-null     float64
 3   Selic ano anterior   22 non-null     float64
 4   Selic inicio do ano  23 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 1.2 KB


### Dados reais

Pesquisando os indicadores que serão analisados, cheguei nos códigos 13552 (IPCA 12 meses) e 432 (Meta Selic).

In [12]:
# Captura dos dados na API
codigo_ipca_12_meses = 13522
data_inicial = '01012000'
data_final = '31122099'
ipca_12_meses_json =    'http://api.bcb.gov.br/dados/serie/' \
                        f'bcdata.sgs.{codigo_ipca_12_meses}/dados?' \
                        f'formato=json&dataInicial={data_inicial}&dataFinal={data_final}'

codigo_meta_selic = 432
meta_selic_json =   'http://api.bcb.gov.br/dados/serie/' \
                    f'bcdata.sgs.{codigo_meta_selic}/dados?' \
                    f'formato=json&dataInicial={data_inicial}&dataFinal={data_final}'

# Inclusão dos dados em DataFrames
ipca_12_meses = pd.read_json(ipca_12_meses_json).set_index('data')
meta_selic = pd.read_json(meta_selic_json).set_index('data')

# Unindo os dados obtidos
df_indicadores_reais = pd.concat([ipca_12_meses, meta_selic], axis = 1, join = 'inner').reset_index()
df_indicadores_reais.reset_index()
df_indicadores_reais.columns = ['Data', 'IPCA', 'MetaSelic']
df_indicadores_reais['Ano'] = df_indicadores_reais.Data.apply(lambda x: int(x.rsplit('/',1)[1]))
df_indicadores_reais.sample(10)

,Data,IPCA,MetaSelic,Ano
146,01/03/2012,5.24,10.50,2012
47,01/12/2003,9.30,17.50,2003
202,01/11/2016,6.99,14.00,2016
57,01/10/2004,6.86,16.25,2004
102,01/07/2008,6.37,12.25,2008
53,01/06/2004,6.06,16.00,2004
73,01/02/2006,5.51,17.25,2006
3,01/04/2000,6.77,18.50,2000
263,01/12/2021,10.06,7.75,2021
181,01/02/2015,7.70,12.25,2015


In [13]:
df_indicadores_reais.shape

(272, 4)

In [14]:
df_indicadores_reais_fechamento_ano = df_indicadores_reais[df_indicadores_reais.Data.str.contains('/12/')]
df_indicadores_reais_fechamento_ano

,Data,IPCA,MetaSelic,Ano
11,01/12/2000,5.97,16.50,2000
23,01/12/2001,7.67,19.00,2001
35,01/12/2002,12.53,22.00,2002
47,01/12/2003,9.30,17.50,2003
59,01/12/2004,7.60,17.25,2004
71,01/12/2005,5.69,18.50,2005
83,01/12/2006,3.14,13.25,2006
95,01/12/2007,4.46,11.25,2007
107,01/12/2008,5.90,13.75,2008
119,01/12/2009,4.31,8.75,2009


In [15]:
df_indicadores_reais_fechamento_ano.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 263
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Data       22 non-null     object 
 1   IPCA       22 non-null     float64
 2   MetaSelic  22 non-null     float64
 3   Ano        22 non-null     int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 880.0+ bytes
